In [1]:
!pip install fastapi uvicorn pyngrok bs4
!pip install google-search-results
%pip install -U langchain-community langgraph langchain-anthropic tavily-python langgraph-checkpoint-sqlite
!pip install gradio
!pip install langchain-openai
!pip install langchain
!pip install -U langchain-community
!pip install langchain together
!pip install  together

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32009 sha256=6fca5be0c36ac06f194d785e75d27907bb9f320ebcb780d674902818008b2ee8
  Stored in directory: /root/.cache/pip/wheels/d3/b2/c3/03302d12bb44a2cdff3c9371f31b72c0c4e84b8d2285eeac53
Successfully built google-search-results
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.5/199.5 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.5/409.5 kB 22.6 MB/s eta 0:00:00
 

In [1]:
from fastapi import FastAPI,UploadFile,File,HTTPException
import uvicorn
from pyngrok import ngrok
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import string
from langchain.agents import load_tools
from together import Together
from langchain_openai import ChatOpenAI
import requests
from bs4 import BeautifulSoup
import os
from langchain.agents import Tool, initialize_agent, AgentType
from langchain.chat_models import ChatOpenAI
!ngrok config add-authtoken 2opUHOetl6GsvbpPlhrVJiK0bQq_d4jPqb3XbX8dpenLgi47

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
 data=pd.read_csv('/content/covid_toy.csv')

In [ ]:
app = FastAPI()

@app.post("/upload-file/")
async def upload_file(file: UploadFile = File(...)):
    try:
        if file.filename.endswith('.csv'):
            data = pd.read_csv(file.file)
            analyse(data)
        elif file.filename.endswith('.xlsx'):
            data = pd.read_excel(file.file)
            analyse(data)
        else:
            raise HTTPException(status_code=400, detail="File format not supported")


        return {"message": "File uploaded and processed successfully"}
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

def analyse(data):

  os.environ['TOGETHER_API_KEY'] = "666e8dc46b45912071558831227d603654035a8f70c3e9f0a41f6f0f9661ad98"
  serpapi='db7f6d98016782c4c1d09f3b05d83bc82b64ec5f64a9473c573b111c723a3f64'
  model = ChatOpenAI(
      base0_url="https://api.together.xyz/v1",
      api_key=os.environ["TOGETHER_API_KEY"],
      model="mistralai/Mixtral-8x7B-Instruct-v0.1",
  )

  def chatgpt_tool(query: str) -> str:
      """Use ChatGPT to answer general knowledge questions."""
      response = model.predict(query)
      return response
  # Import Tool from langchain.agents
  tools = [
      Tool(name="ChatGPT", func=chatgpt_tool, description="Answers general questions using ChatGPT.")
  ]

  agent = initialize_agent(
      tools,
      model,
      agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
      verbose=True,
  )
  prompt=f'''Given the information on the dataset : The columns of the dataset {list(data.columns)} , first 5 rows of the dataset {data.head()} , Mode values of the dataset {data.mode()} and the null values of the dataset {data.isna().sum()} . Give the short description of the dataset and
  also suggest the best method to handle the null values for this dataset'''
  out=agent.run(prompt)
  print(out)
  value=[]
  num=[i for i in range(1,11)]
  floats=[float(i) for i in num]
  symbols =[',','/',';',':','%' ,'$','&','<','>','*','^','(',')','#','@','!','-1','?']
  alphabet=list(string.ascii_lowercase)
  if isinstance(data[i].dtype , object):
            x=data[i].mode()[0]
            value.append(x)
            index=len(data)//5
            for m in range (1,6):
                x=data.iloc[index*m][i]
                print(x)
                value.append(x)
            l=[]
            count=0
            for m in value:
              x=list(m)
              for j in x:
                if (j not in alphabet):
                    l.append(j)
              if(len(x)==len(l)):
                count+=1
                l=[]
            if(count==6):
              for m in value:
                x=list(m)
                for j in x:
                      if(j in symbols):
                          data[i]=data[i].apply(lambda x: x.replace(j,''))
              data[i]=data[i].astype(float)
              print(data[i].dtype)

  for i in symbols:
      data.replace({i: np.nan}, inplace=True)
  col=list(data.columns[data.isnull().any()])
  for i in col:
      print(data[i].dtype)
      if any(letter in str(val) for val in data[i] for letter in alphabet):
          x=int(input(f'''For column {i} Enter the choice for null value handling\n1. Deletion\n2. Fill with Mode value\n3. Fill with Median\n4.Fill with Mean\n.5 Fill with the mean of last 10 non-null values\n6.5 Fill with the mean of last 20 non-null values'''))
          if(x==1):
                  data[i].dropna(inplace=True)
          if(x==2):
                  data[i].fillna(data.mode()[0],inplace=True)
          if(x==3):
                  data[i].fillna(data.median(),inplace=True)
          if(x==4):
                  data[i].fillna(data.mean(),inplace=True)
          if(x==5):
                  data[i].fillna(data.rolling(10,min_periods=1).mean(),inplace=True)
          if(x==6):
                  data[i].fillna(data.rolling(20,min_periods=1).mean(),inplace=True)


      else:
          x=int(input(f'''For column {i} Enter the choice for null value handling\n1. Deletion\n2. Fill with Mode value\n3. Fill with Median\n4.Fill with Mean\n.5 Fill with the mean of last 10 non-null values\n6.5 Fill with the mean of last 20 non-null values'''))
          if(x==1):
                  data[i].dropna(inplace=True)
          if(x==2):
                  data[i].fillna(data.mode()[0],inplace=True)
          if(x==3):
                  data[i].fillna(data.median(),inplace=True)














In [ ]:
import threading

# Function to start the FastAPI app
def start_app():
    uvicorn.run(app, host="0.0.0.0", port=8100)

# Start FastAPI app in a thread
thread = threading.Thread(target=start_app)
thread.start()

# Open a tunnel using ngrok
public_url = ngrok.connect(8100)
print("FastAPI is running on:", public_url)


INFO:     Started server process [2839]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8100 (Press CTRL+C to quit)


FastAPI is running on: NgrokTunnel: "https://333e-34-143-139-107.ngrok-free.app" -> "http://localhost:8100"


In [4]:
import re
data=pd.read_csv('/content/covid_toy.csv')
def analyse(data):

  os.environ['TOGETHER_API_KEY'] = "666e8dc46b45912071558831227d603654035a8f70c3e9f0a41f6f0f9661ad98"
  serpapi='db7f6d98016782c4c1d09f3b05d83bc82b64ec5f64a9473c573b111c723a3f64'
  model = ChatOpenAI(
      base_url="https://api.together.xyz/v1",
      api_key=os.environ["TOGETHER_API_KEY"],
      model="meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo",
  )

  def chatgpt_tool(query: str) -> str:
      """Use ChatGPT to answer general knowledge questions."""
      response = model.predict(query)
      return response
  # Import Tool from langchain.agents
  tools = [
      Tool(name="ChatGPT", func=chatgpt_tool, description="Answers general questions using ChatGPT.")
  ]

  agent = initialize_agent(
      tools,
      model,
      agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
      verbose=True,
  )
  prompt=f"Describe the dataset {data} using its column names, the first 5 rows, mode values, and null value counts. Suggest the best method to handle the null values."
  out=agent.run(input=prompt,handle_parsing_errors=True)
  print(out)
  value=[]
  num=[i for i in range(1,11)]
  floats=[float(i) for i in num]
  symbols =[',','/',';',':','%' ,'$','&','<','>','*','^','(',')','#','@','!','-1','?']
  alphabet=list(string.ascii_lowercase)
  for i in data.columns:
    if isinstance(data[i].dtype , object):
              x=data[i].mode()[0]
              value.append(x)
              index=len(data)//5
              for m in range (1,6):
                  if index * m < len(data):
                    x=data.iloc[index*m][i]
                  else:
                    x=data.iloc[len(data)-1][i]

                  value.append(x)
              l=[]
              count=0
              for m in value:
                x=list(str(m))
                for j in x:
                  if (j not in alphabet):
                      l.append(j)
                if(len(x)==len(l)):
                  count+=1
                  l=[]
              if(count==6):
                for m in value:
                  x=list(str(m))
                  for j in x:
                        if(j in symbols):
                            data[i]=data[i].apply(lambda x: x.replace(j,''))
                data[i]=data[i].astype(float)
                print(data[i].dtype)

  for i in symbols:
      data.replace({i: np.nan}, inplace=True)
  col=list(data.columns[data.isnull().any()])
  for i in col:
      print(data[i].dtype)
      if any(letter in str(val) for val in data[i] for letter in alphabet):
          x=int(input(f'''For column {i} Enter the choice for null value handling\n1. Deletion\n2. Fill with Mode value\n3. Fill with Median\n4.Fill with Mean\n5. Fill with the mean of last 10 non-null values\n6.Fill with the mean of last 20 non-null values'''))
          if(x==1):
                  data[i].dropna(inplace=True)
          if(x==2):
                  data[i].fillna(data[i].mode()[0],inplace=True)
          if(x==3):
                  data[i].fillna(data[i].median(),inplace=True)
          if(x==4):
                  data[i].fillna(data[i].mean(),inplace=True)
          if(x==5):
                  data[i].fillna(data[i].rolling(10,min_periods=1).mean(),inplace=True)
          if(x==6):
                  data[i].fillna(data[i].rolling(20,min_periods=1).mean(),inplace=True)


      else:
          x=int(input(f'''For column {i} Enter the choice for null value handling\n1. Deletion\n2. Fill with Mode value\n3. Fill with Median\n4.Fill with Mean\n5. Fill with the mean of last 10 non-null values\n6. Fill with the mean of last 20 non-null values'''))
          if(x==1):
                  data[i].dropna(inplace=True)
          if(x==2):
                  data[i].fillna(data[i].mode()[0],inplace=True)
          if(x==3):
                  data[i].fillna(data[i].median(),inplace=True)
  # data modeling
  prompt=f'Given the dataset {data} identify the target column and also identify which type of regression(linear/logistic) will work for this kind of data.Please print the column name as mentioned in the {data.columns} given and the regression type nothing else '
  output=agent.run(input=prompt,handle_parsing_errors=True)
  print(output)
  words= re.split(r'[,\s\']+', output)
  print(words)
  for i in data.columns:
    if(i in words):
      print(i)
      target=data.pop(i)
    else:
      pass
  if('linear' in words) or ("Linear" in words):
    d={}
    for i in data.columns:
      if(data[i].dtype=='object'):
        d[i]=data[i].unique()
    print(d)
    for i in d.keys():
      if(len(d[i])==2):
        data[i]=data[i].apply(lambda x: 0 if(x==d[i][0]) else 1)
      else:
        x=pd.get_dummies(data[i],dtype=int,prefix=i)
        data=pd.concat([data,x],axis=1)
        data.drop(i,axis=1,inplace=True)
        x=0
    m=agent.run(input=f"Which among the 'Normalization/Standardization' will be best suited for {data}, use only 20 characters for answer",handle_parsing_errors=True)
    print(m)
    x=m.split()
    col=[]
    if('Normalization' in x):
      from sklearn.preprocessing import MinMaxScaler
      sc=MinMaxScaler()
      for i in data.columns:
        if(data[i].dtype=='object'):
          pass
        else:
           col.append(i)
      data[col]=sc.fit_transform(data[col])
    else:
      from sklearn.preprocessing import StandardScaler
      sc=StandardScaler()
      for i in data.columns:
        if(data[i].dtype=='object'):
          pass
        else:
          col.append(i)
      data[col]=sc.fit_transform(data[col])
    from sklearn.feature_selection import RFE
    from sklearn.model_selection import train_test_split
    from sklearn.linear_model import LinearRegression
    from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error,root_mean_squared_error
    x_train,x_test,y_train,y_test=train_test_split(data,target,test_size=0.2,random_state=42)
    lr=LinearRegression()
    lr.fit(x_train,y_train)
    y_pred=lr.predict(x_test)
    print('r2 : ' ,r2_score(y_test,y_pred))
    print('MSE : ' ,mean_squared_error(y_test,y_pred))
    print('RMSE : ' ,root_mean_squared_error(y_test,y_pred))
    print('MAE : ' ,mean_absolute_error(y_test,y_pred))
    print(data)

  elif('logistic' in words) or ("Logistic" in words):
    d={}
    for i in data.columns:
      if(data[i].dtype=='object'):
        d[i]=data[i].unique()
    print(d)
    for i in d.keys():
      if(len(d[i])==2):
        data[i]=data[i].apply(lambda x: 0 if(x==d[i][0]) else 1)
      else:
        x=pd.get_dummies(data[i],dtype=int,prefix=i)
        data=pd.concat([data,x],axis=1)
        data.drop(i,axis=1,inplace=True)
        x=0
    m=agent.run(input=f"answer in 1 word. Which among the 'Normalization/Standardization' will be best suited for {data}",handle_parsing_errors=True)
    print(m)
    x=m.split()
    col=[]
    if('Normalization' in x):
      from sklearn.preprocessing import MinMaxScaler
      sc=MinMaxScaler()
      for i in data.columns:
        if(data[i].dtype=='object'):
          pass
        else:
          col.append(i)
      data[col]=sc.fit_transform(data[col])
    else:
      from sklearn.preprocessing import StandardScaler
      sc=StandardScaler()
      for i in data.columns:
        if(data[i].dtype=='object'):
          pass
        else:
          col.append(i)
      data[col]=sc.fit_transform(data[col])
    from sklearn.feature_selection import RFE
    from sklearn.model_selection import train_test_split
    from sklearn.linear_model import LogisticRegression
    from sklearn.metrics import confusion_matrix,accuracy_score,classification_report
    x_train,x_test,y_train,y_test=train_test_split(data,target,test_size=0.2,random_state=42)
    accuracy_list={}
    x=int(input(f'''Enter the choice for classification model\n1. Logistic Regression\n2. Naive Bayes \n3.K-Neighbour Model\n4.Decision Tree\n5. No choice'''))
    if(x==1):
      lr=LogisticRegression()
      lr.fit(x_train,y_train)
      y_pred=lr.predict(x_test)
      print(confusion_matrix(y_test,y_pred))
      accuracy_list['Logistic_Regression']=accuracy_score(y_test,y_pred)
      print(accuracy_score(y_test,y_pred))
      print(classification_report(y_test,y_pred))
    if(x==2):
      from sklearn.naive_bayes import GaussianNB
      nb=GaussianNB()
      nb.fit(x_train,y_train)
      y_pred=nb.predict(x_test)
      print(confusion_matrix(y_test,y_pred))
      accuracy_list['Naive_Bayes']=accuracy_score(y_test,y_pred)
      print(accuracy_score(y_test,y_pred))
      print(classification_report(y_test,y_pred))
    if(x==3):
      from sklearn.neighbors import KNeighborsClassifier
      knn=KNeighborsClassifier()
      knn.fit(x_train,y_train)
      y_pred=knn.predict(x_test)
      print(confusion_matrix(y_test,y_pred))
      accuracy_list['KNN']=accuracy_score(y_test,y_pred)
      print(accuracy_score(y_test,y_pred))
      print(classification_report(y_test,y_pred))
    if(x==4):
      from sklearn.tree import DecisionTreeClassifier
      dt=DecisionTreeClassifier()
      dt.fit(x_train,y_train)
      y_pred=dt.predict(x_test)
      print(confusion_matrix(y_test,y_pred))
      accuracy_list['Decision_tree']=accuracy_score(y_test,y_pred)
      print(accuracy_score(y_test,y_pred))
      print(classification_report(y_test,y_pred))
    if(x==5):
      lr=LogisticRegression()
      lr.fit(x_train,y_train)
      y_pred=lr.predict(x_test)
      accuracy_list['Logistic_Regression']=accuracy_score(y_test,y_pred)
      from sklearn.naive_bayes import GaussianNB
      nb=GaussianNB()
      nb.fit(x_train,y_train)
      y_pred=nb.predict(x_test)
      accuracy_list['Naive_Bayes']=accuracy_score(y_test,y_pred)
      from sklearn.neighbors import KNeighborsClassifier
      knn=KNeighborsClassifier()
      knn.fit(x_train,y_train)
      y_pred=knn.predict(x_test)
      accuracy_list['KNN']=accuracy_score(y_test,y_pred)
      from sklearn.tree import DecisionTreeClassifier
      dt=DecisionTreeClassifier()
      dt.fit(x_train,y_train)
      y_pred=dt.predict(x_test)
      accuracy_list['Decision_tree']=accuracy_score(y_test,y_pred)
      sorted_dict = dict(sorted(accuracy_list.items(), key=lambda item: item[1]))
      print("Sorted Dictionary:", sorted_dict)
      i=sorted_dict.keys()[0]
      if(i=='Logistic_Regression'):
          lr=LogisticRegression()
          lr.fit(x_train,y_train)
          y_pred=lr.predict(x_test)
          print(confusion_matrix(y_test,y_pred))
          print(accuracy_score(y_test,y_pred))
          print(classification_report(y_test,y_pred))
      if(i=='Naive_Bayes'):
          from sklearn.naive_bayes import GaussianNB
          nb=GaussianNB()
          nb.fit(x_train,y_train)
          y_pred=nb.predict(x_test)
          print(confusion_matrix(y_test,y_pred))
          print(accuracy_score(y_test,y_pred))
          print(classification_report(y_test,y_pred))
      if(i=='KNN'):
          from sklearn.neighbors import KNeighborsClassifier
          knn=KNeighborsClassifier()
          knn.fit(x_train,y_train)
          y_pred=knn.predict(x_test)
          print(confusion_matrix(y_test,y_pred))
          print(accuracy_score(y_test,y_pred))
          print(classification_report(y_test,y_pred))
      if(i=='Decision_tree'):
          from sklearn.tree import DecisionTreeClassifier
          dt=DecisionTreeClassifier()
          dt.fit(x_train,y_train)
          y_pred=dt.predict(x_test)
          print(confusion_matrix(y_test,y_pred))
          print(accuracy_score(y_test,y_pred))
          print(classification_report(y_test,y_pred))
    print(data)





In [5]:
analyse(data)



> Entering new AgentExecutor chain...
Thought: The problem is asking me to describe a dataset with its column names, the first 5 rows, mode values, and null value counts. It also wants me to suggest the best method to handle the null values.

Action: ChatGPT
Action Input: Describe the dataset with column names, first 5 rows, mode values, and null value counts. Suggest the best method to handle the null values.
Observation: I'll provide a general description of a dataset, but please note that I'll be using a sample dataset for the purpose of illustration. Let's assume we have a dataset called `students` with the following characteristics.

**Dataset Description:**

The `students` dataset contains information about 1000 students, including their demographics, academic performance, and extracurricular activities. The dataset has the following columns:

| Column Name | Data Type | Description |
| --- | --- | --- |
| `StudentID` | Integer | Unique identifier for each student |
| `Name` | 

<ipython-input-4-08399f11f980>:81: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data[i].fillna(data[i].mean(),inplace=True)




> Entering new AgentExecutor chain...
Thought: To identify the target column and the type of regression, we need to analyze the given dataset. The target column is usually the column that we want to predict. In this case, the column 'has_covid' seems to be the target column as it contains Yes/No values indicating whether a person has Covid or not. 

For the type of regression, since the target column 'has_covid' contains categorical values (Yes/No), we can conclude that this is a classification problem. However, in the context of regression, we typically talk about logistic regression when dealing with binary classification problems.

Action: ChatGPT
Action Input: Which type of regression will work for a binary classification problem like 'has_covid' in the given dataset?
Observation: For a binary classification problem like 'has_covid', you would typically use logistic regression. Logistic regression is a type of regression analysis used to model a binary outcome (0/1, yes/no, etc.)

In [20]:
import sys
import trace
import nbformat

# Initialize a tracer
tracer = trace.Trace(count=False, trace=True)

# Function to log executed lines
def trace_function(func):
    def wrapper(*args, **kwargs):
        sys.settrace(tracer.runfunc)
        result = func(*args, **kwargs)
        sys.settrace(None)
        return result
    return wrapper

# Code Execution

def execute_model():
    # Place your provided code here as a function
    import re
    import os
    import string
    import numpy as np
    import pandas as pd
    from langchain.agents import initialize_agent, Tool
    from langchain.agents import AgentType

    data = pd.read_csv('/content/covid_toy.csv')

    # Define the provided code as is
    def analyse(data):

      os.environ['TOGETHER_API_KEY'] = "666e8dc46b45912071558831227d603654035a8f70c3e9f0a41f6f0f9661ad98"
      serpapi='db7f6d98016782c4c1d09f3b05d83bc82b64ec5f64a9473c573b111c723a3f64'
      model = ChatOpenAI(
          base_url="https://api.together.xyz/v1",
          api_key=os.environ["TOGETHER_API_KEY"],
          model="mistralai/Mixtral-8x7B-Instruct-v0.1",
      )

      def chatgpt_tool(query: str) -> str:
          """Use ChatGPT to answer general knowledge questions."""
          response = model.predict(query)
          return response
      # Import Tool from langchain.agents
      tools = [
          Tool(name="ChatGPT", func=chatgpt_tool, description="Answers general questions using ChatGPT.")
      ]

      agent = initialize_agent(
          tools,
          model,
          agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
          verbose=True,
      )
      prompt=f"Describe the dataset {data} using its column names, the first 5 rows, mode values, and null value counts. Suggest the best method to handle the null values."
      out=agent.run(input=prompt,handle_parsing_errors=True)
      print(out)
      value=[]
      num=[i for i in range(1,11)]
      floats=[float(i) for i in num]
      symbols =[',','/',';',':','%' ,'$','&','<','>','*','^','(',')','#','@','!','-1','?']
      alphabet=list(string.ascii_lowercase)
      for i in data.columns:
        if isinstance(data[i].dtype , object):
                  x=data[i].mode()[0]
                  value.append(x)
                  index=len(data)//5
                  for m in range (1,6):
                      if index * m < len(data):
                        x=data.iloc[index*m][i]
                      else:
                        x=data.iloc[len(data)-1][i]

                      value.append(x)
                  l=[]
                  count=0
                  for m in value:
                    x=list(str(m))
                    for j in x:
                      if (j not in alphabet):
                          l.append(j)
                    if(len(x)==len(l)):
                      count+=1
                      l=[]
                  if(count==6):
                    for m in value:
                      x=list(str(m))
                      for j in x:
                            if(j in symbols):
                                data[i]=data[i].apply(lambda x: x.replace(j,''))
                    data[i]=data[i].astype(float)
                    print(data[i].dtype)

      for i in symbols:
          data.replace({i: np.nan}, inplace=True)
      col=list(data.columns[data.isnull().any()])
      for i in col:
          print(data[i].dtype)
          if any(letter in str(val) for val in data[i] for letter in alphabet):
              x=int(input(f'''For column {i} Enter the choice for null value handling\n1. Deletion\n2. Fill with Mode value\n3. Fill with Median\n4.Fill with Mean\n5. Fill with the mean of last 10 non-null values\n6.Fill with the mean of last 20 non-null values'''))
              if(x==1):
                      data[i].dropna(inplace=True)
              if(x==2):
                      data[i].fillna(data[i].mode()[0],inplace=True)
              if(x==3):
                      data[i].fillna(data[i].median(),inplace=True)
              if(x==4):
                      data[i].fillna(data[i].mean(),inplace=True)
              if(x==5):
                      data[i].fillna(data[i].rolling(10,min_periods=1).mean(),inplace=True)
              if(x==6):
                      data[i].fillna(data[i].rolling(20,min_periods=1).mean(),inplace=True)


          else:
              x=int(input(f'''For column {i} Enter the choice for null value handling\n1. Deletion\n2. Fill with Mode value\n3. Fill with Median\n4.Fill with Mean\n5. Fill with the mean of last 10 non-null values\n6. Fill with the mean of last 20 non-null values'''))
              if(x==1):
                      data[i].dropna(inplace=True)
              if(x==2):
                      data[i].fillna(data[i].mode()[0],inplace=True)
              if(x==3):
                      data[i].fillna(data[i].median(),inplace=True)
      # data modeling
      prompt=f'From the dataset {data}, identify the target column and specify whether to use linear or logistic regression. Only provide the column name and regression type.'
      output=agent.run(input=prompt,handle_parsing_errors=True)
      print(output)
      words= re.split(r'[,\s\'\"]+', output)
      print(words)
      for i in data.columns:
        if(i in words):
          print(i)
          target=data.pop(i)
        else:
          pass
      if('linear' in words) or ("Linear" in words):
        d={}
        for i in data.columns:
          if(data[i].dtype=='object'):
            d[i]=data[i].unique()
        print(d)
        for i in d.keys():
          if(len(d[i])==2):
            data[i]=data[i].apply(lambda x: 0 if(x==d[i][0]) else 1)
          else:
            x=pd.get_dummies(data[i],dtype=int,prefix=i)
            data=pd.concat([data,x],axis=1)
            data.drop(i,axis=1,inplace=True)
            x=0
        m=agent.run(input=f"Which among the 'Normalization/Standardization' will be best suited for {data}, use only 20 characters for answer",handle_parsing_errors=True)
        print(m)
        x=m.split()
        col=[]
        if('Normalization' in x):
          from sklearn.preprocessing import MinMaxScaler
          sc=MinMaxScaler()
          for i in data.columns:
            if(data[i].dtype=='object'):
              pass
            else:
              col.append(i)
          data[col]=sc.fit_transform(data[col])
        else:
          from sklearn.preprocessing import StandardScaler
          sc=StandardScaler()
          for i in data.columns:
            if(data[i].dtype=='object'):
              pass
            else:
              col.append(i)
          data[col]=sc.fit_transform(data[col])
        from sklearn.feature_selection import RFE
        from sklearn.model_selection import train_test_split
        from sklearn.linear_model import LinearRegression
        from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error,root_mean_squared_error
        x_train,x_test,y_train,y_test=train_test_split(data,target,test_size=0.2,random_state=42)
        lr=LinearRegression()
        lr.fit(x_train,y_train)
        y_pred=lr.predict(x_test)
        print('r2 : ' ,r2_score(y_test,y_pred))
        print('MSE : ' ,mean_squared_error(y_test,y_pred))
        print('RMSE : ' ,root_mean_squared_error(y_test,y_pred))
        print('MAE : ' ,mean_absolute_error(y_test,y_pred))
        print(data)

      elif('logistic' in words) or ("Logistic" in words):
        d={}
        for i in data.columns:
          if(data[i].dtype=='object'):
            d[i]=data[i].unique()
        print(d)
        for i in d.keys():
          if(len(d[i])==2):
            data[i]=data[i].apply(lambda x: 0 if(x==d[i][0]) else 1)
          else:
            x=pd.get_dummies(data[i],dtype=int,prefix=i)
            data=pd.concat([data,x],axis=1)
            data.drop(i,axis=1,inplace=True)
            x=0
        m=agent.run(input=f"answer in 1 word. Which among the 'Normalization/Standardization' will be best suited for {data}",handle_parsing_errors=True)
        print(m)
        x=m.split()
        col=[]
        if('Normalization' in x):
          from sklearn.preprocessing import MinMaxScaler
          sc=MinMaxScaler()
          for i in data.columns:
            if(data[i].dtype=='object'):
              pass
            else:
              col.append(i)
          data[col]=sc.fit_transform(data[col])
        else:
          from sklearn.preprocessing import StandardScaler
          sc=StandardScaler()
          for i in data.columns:
            if(data[i].dtype=='object'):
              pass
            else:
              col.append(i)
          data[col]=sc.fit_transform(data[col])
        from sklearn.feature_selection import RFE
        from sklearn.model_selection import train_test_split
        from sklearn.linear_model import LogisticRegression
        from sklearn.metrics import confusion_matrix,accuracy_score,classification_report
        x_train,x_test,y_train,y_test=train_test_split(data,target,test_size=0.2,random_state=42)
        accuracy_list={}
        x=int(input(f'''Enter the choice for classification model\n1. Logistic Regression\n2. Naive Bayes \n3.K-Neighbour Model\n4.Decision Tree\n5. No choice'''))
        if(x==1):
          lr=LogisticRegression()
          lr.fit(x_train,y_train)
          y_pred=lr.predict(x_test)
          print(confusion_matrix(y_test,y_pred))
          accuracy_list['Logistic_Regression']=accuracy_score(y_test,y_pred)
          print(accuracy_score(y_test,y_pred))
          print(classification_report(y_test,y_pred))
        if(x==2):
          from sklearn.naive_bayes import GaussianNB
          nb=GaussianNB()
          nb.fit(x_train,y_train)
          y_pred=nb.predict(x_test)
          print(confusion_matrix(y_test,y_pred))
          accuracy_list['Naive_Bayes']=accuracy_score(y_test,y_pred)
          print(accuracy_score(y_test,y_pred))
          print(classification_report(y_test,y_pred))
        if(x==3):
          from sklearn.neighbors import KNeighborsClassifier
          knn=KNeighborsClassifier()
          knn.fit(x_train,y_train)
          y_pred=knn.predict(x_test)
          print(confusion_matrix(y_test,y_pred))
          accuracy_list['KNN']=accuracy_score(y_test,y_pred)
          print(accuracy_score(y_test,y_pred))
          print(classification_report(y_test,y_pred))
        if(x==4):
          from sklearn.tree import DecisionTreeClassifier
          dt=DecisionTreeClassifier()
          dt.fit(x_train,y_train)
          y_pred=dt.predict(x_test)
          print(confusion_matrix(y_test,y_pred))
          accuracy_list['Decision_tree']=accuracy_score(y_test,y_pred)
          print(accuracy_score(y_test,y_pred))
          print(classification_report(y_test,y_pred))
        if(x==5):
          lr=LogisticRegression()
          lr.fit(x_train,y_train)
          y_pred=lr.predict(x_test)
          accuracy_list['Logistic_Regression']=accuracy_score(y_test,y_pred)
          from sklearn.naive_bayes import GaussianNB
          nb=GaussianNB()
          nb.fit(x_train,y_train)
          y_pred=nb.predict(x_test)
          accuracy_list['Naive_Bayes']=accuracy_score(y_test,y_pred)
          from sklearn.neighbors import KNeighborsClassifier
          knn=KNeighborsClassifier()
          knn.fit(x_train,y_train)
          y_pred=knn.predict(x_test)
          accuracy_list['KNN']=accuracy_score(y_test,y_pred)
          from sklearn.tree import DecisionTreeClassifier
          dt=DecisionTreeClassifier()
          dt.fit(x_train,y_train)
          y_pred=dt.predict(x_test)
          accuracy_list['Decision_tree']=accuracy_score(y_test,y_pred)
          sorted_dict = dict(sorted(accuracy_list.items(), key=lambda item: item[1]))
          print("Sorted Dictionary:", sorted_dict)
          i = list(sorted_dict.keys())[0]
          if(i=='Logistic_Regression'):
              lr=LogisticRegression()
              lr.fit(x_train,y_train)
              y_pred=lr.predict(x_test)
              print(confusion_matrix(y_test,y_pred))
              print(accuracy_score(y_test,y_pred))
              print(classification_report(y_test,y_pred))
          if(i=='Naive_Bayes'):
              from sklearn.naive_bayes import GaussianNB
              nb=GaussianNB()
              nb.fit(x_train,y_train)
              y_pred=nb.predict(x_test)
              print(confusion_matrix(y_test,y_pred))
              print(accuracy_score(y_test,y_pred))
              print(classification_report(y_test,y_pred))
          if(i=='KNN'):
              from sklearn.neighbors import KNeighborsClassifier
              knn=KNeighborsClassifier()
              knn.fit(x_train,y_train)
              y_pred=knn.predict(x_test)
              print(confusion_matrix(y_test,y_pred))
              print(accuracy_score(y_test,y_pred))
              print(classification_report(y_test,y_pred))
          if(i=='Decision_tree'):
              from sklearn.tree import DecisionTreeClassifier
              dt=DecisionTreeClassifier()
              dt.fit(x_train,y_train)
              y_pred=dt.predict(x_test)
              print(confusion_matrix(y_test,y_pred))
              print(accuracy_score(y_test,y_pred))
              print(classification_report(y_test,y_pred))
        print(data)





    analyse(data)

execute_model()

# Create nbformat notebook
nb = nbformat.v4.new_notebook()

# Append executed lines to notebook cells
executed_lines = tracer.results()

# Convert CoverageResults to a list of executed lines
# This is a placeholder and needs to be adapted to how your tracer stores the lines
executed_lines_list = [str(line) for line in executed_lines.executed_lines()]  # Example using hypothetical executed_lines.executed_lines() method

nb.cells.append(nbformat.v4.new_code_cell('\n'.join(executed_lines_list)))

# Save as .ipynb file
with open("executed_lines.ipynb", "w") as f:
    nbformat.write(nb, f)

# Save as .ipynb file
with open("executed_lines.ipynb", "w") as f:
    nbformat.write(nb, f)




> Entering new AgentExecutor chain...
 To answer this question, I need to gather information about the dataset using the column names, the first 5 rows, mode values, and null value counts. I will use the ChatGPT tool to find out the mode values and null count.

Action: ChatGPT
Action Input: What is the mode value for age, gender, fever, cough, city, and has_covid in the given dataset? Also, how many null values are there in the dataset?

Let's wait for the response from ChatGPT and continue from there.

Observation:  I'm assuming you'd like me to provide a response as if I were asking the question. Here is a possible way the conversation could continue:

As a helpful assistant, I would need the dataset to be able to provide you with the mode values for age, gender, fever, cough, city, and has_covid, as well as the number of null values. Could you please share the dataset with me?

However, I can explain how to find these values for a general understanding. Mode value is the most freq

<ipython-input-20-8808167f03f5>:110: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data[i].fillna(data[i].mean(),inplace=True)




> Entering new AgentExecutor chain...
 The target column in this dataset is "has_covid". This is a binary column where the values are either "Yes" or "No", indicating whether a person has COVID-19 or not. For such a target variable, we use logistic regression. Therefore, the answer is "has_covid" and "logistic regression".

Final Answer: has_covid, logistic regression

> Finished chain.
has_covid, logistic regression
['has_covid', 'logistic', 'regression']
has_covid
{'gender': array(['Male', 'Female'], dtype=object), 'cough': array(['Mild', 'Strong'], dtype=object), 'city': array(['Kolkata', 'Delhi', 'Mumbai', 'Bangalore'], dtype=object)}


> Entering new AgentExecutor chain...
 The question asks for a one-word answer, so I need to determine whether normalization or standardization would be more suitable for this dataset. The given dataset contains a mix of numerical and categorical features, with some numerical features (age, fever) having different scales and ranges. Standardizatio

AttributeError: 'CoverageResults' object has no attribute 'executed_lines'

In [3]:
import sys
import trace
import nbformat

# Initialize a tracer
tracer = trace.Trace(count=False, trace=True)

# Function to log executed lines
def trace_function(func):
    def wrapper(*args, **kwargs):
        sys.settrace(tracer.runfunc)
        result = func(*args, **kwargs)
        sys.settrace(None)
        return result
    return wrapper

# Code Execution

def execute_model():
    # Place your provided code here as a function
    # ... (rest of your code) ...
    import re
    import os
    import string
    import numpy as np
    import pandas as pd
    from langchain.agents import initialize_agent, Tool
    from langchain.agents import AgentType

    data = pd.read_csv('/content/covid_toy.csv')

    # Define the provided code as is
    def analyse(data):

      os.environ['TOGETHER_API_KEY'] = "666e8dc46b45912071558831227d603654035a8f70c3e9f0a41f6f0f9661ad98"
      serpapi='db7f6d98016782c4c1d09f3b05d83bc82b64ec5f64a9473c573b111c723a3f64'
      model = ChatOpenAI(
          base_url="https://api.together.xyz/v1",
          api_key=os.environ["TOGETHER_API_KEY"],
          model="mistralai/Mixtral-8x7B-Instruct-v0.1",
      )

      def chatgpt_tool(query: str) -> str:
          """Use ChatGPT to answer general knowledge questions."""
          response = model.predict(query)
          return response
      # Import Tool from langchain.agents
      tools = [
          Tool(name="ChatGPT", func=chatgpt_tool, description="Answers general questions using ChatGPT.")
      ]

      agent = initialize_agent(
          tools,
          model,
          agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
          verbose=True,
      )
      prompt=f"Describe the dataset {data} using its column names, the first 5 rows, mode values, and null value counts. Suggest the best method to handle the null values."
      out=agent.run(input=prompt,handle_parsing_errors=True)
      print(out)
      value=[]
      num=[i for i in range(1,11)]
      floats=[float(i) for i in num]
      symbols =[',','/',';',':','%' ,'$','&','<','>','*','^','(',')','#','@','!','-1','?']
      alphabet=list(string.ascii_lowercase)
      for i in data.columns:
        if isinstance(data[i].dtype , object):
                  x=data[i].mode()[0]
                  value.append(x)
                  index=len(data)//5
                  for m in range (1,6):
                      if index * m < len(data):
                        x=data.iloc[index*m][i]
                      else:
                        x=data.iloc[len(data)-1][i]

                      value.append(x)
                  l=[]
                  count=0
                  for m in value:
                    x=list(str(m))
                    for j in x:
                      if (j not in alphabet):
                          l.append(j)
                    if(len(x)==len(l)):
                      count+=1
                      l=[]
                  if(count==6):
                    for m in value:
                      x=list(str(m))
                      for j in x:
                            if(j in symbols):
                                data[i]=data[i].apply(lambda x: x.replace(j,''))
                    data[i]=data[i].astype(float)
                    print(data[i].dtype)

      for i in symbols:
          data.replace({i: np.nan}, inplace=True)
      col=list(data.columns[data.isnull().any()])
      for i in col:
          print(data[i].dtype)
          if any(letter in str(val) for val in data[i] for letter in alphabet):
              x=int(input(f'''For column {i} Enter the choice for null value handling\n1. Deletion\n2. Fill with Mode value\n3. Fill with Median\n4.Fill with Mean\n5. Fill with the mean of last 10 non-null values\n6.Fill with the mean of last 20 non-null values'''))
              if(x==1):
                      data[i].dropna(inplace=True)
              if(x==2):
                      data[i].fillna(data[i].mode()[0],inplace=True)
              if(x==3):
                      data[i].fillna(data[i].median(),inplace=True)
              if(x==4):
                      data[i].fillna(data[i].mean(),inplace=True)
              if(x==5):
                      data[i].fillna(data[i].rolling(10,min_periods=1).mean(),inplace=True)
              if(x==6):
                      data[i].fillna(data[i].rolling(20,min_periods=1).mean(),inplace=True)


          else:
              x=int(input(f'''For column {i} Enter the choice for null value handling\n1. Deletion\n2. Fill with Mode value\n3. Fill with Median\n4.Fill with Mean\n5. Fill with the mean of last 10 non-null values\n6. Fill with the mean of last 20 non-null values'''))
              if(x==1):
                      data[i].dropna(inplace=True)
              if(x==2):
                      data[i].fillna(data[i].mode()[0],inplace=True)
              if(x==3):
                      data[i].fillna(data[i].median(),inplace=True)
      # data modeling
      prompt=f'From the dataset {data}, identify the target column and specify whether to use linear or logistic regression. Only provide the column name and regression type.'
      output=agent.run(input=prompt,handle_parsing_errors=True)
      print(output)
      words= re.split(r'[,\s\'\"]+', output)
      print(words)
      for i in data.columns:
        if(i in words):
          print(i)
          target=data.pop(i)
        else:
          pass
      if('linear' in words) or ("Linear" in words):
        d={}
        for i in data.columns:
          if(data[i].dtype=='object'):
            d[i]=data[i].unique()
        print(d)
        for i in d.keys():
          if(len(d[i])==2):
            data[i]=data[i].apply(lambda x: 0 if(x==d[i][0]) else 1)
          else:
            x=pd.get_dummies(data[i],dtype=int,prefix=i)
            data=pd.concat([data,x],axis=1)
            data.drop(i,axis=1,inplace=True)
            x=0
        m=agent.run(input=f"Which among the 'Normalization/Standardization' will be best suited for {data}, use only 20 characters for answer",handle_parsing_errors=True)
        print(m)
        x=m.split()
        col=[]
        if('Normalization' in x):
          from sklearn.preprocessing import MinMaxScaler
          sc=MinMaxScaler()
          for i in data.columns:
            if(data[i].dtype=='object'):
              pass
            else:
              col.append(i)
          data[col]=sc.fit_transform(data[col])
        else:
          from sklearn.preprocessing import StandardScaler
          sc=StandardScaler()
          for i in data.columns:
            if(data[i].dtype=='object'):
              pass
            else:
              col.append(i)
          data[col]=sc.fit_transform(data[col])
        from sklearn.feature_selection import RFE
        from sklearn.model_selection import train_test_split
        from sklearn.linear_model import LinearRegression
        from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error,root_mean_squared_error
        x_train,x_test,y_train,y_test=train_test_split(data,target,test_size=0.2,random_state=42)
        lr=LinearRegression()
        lr.fit(x_train,y_train)
        y_pred=lr.predict(x_test)
        print('r2 : ' ,r2_score(y_test,y_pred))
        print('MSE : ' ,mean_squared_error(y_test,y_pred))
        print('RMSE : ' ,root_mean_squared_error(y_test,y_pred))
        print('MAE : ' ,mean_absolute_error(y_test,y_pred))
        print(data)

      elif('logistic' in words) or ("Logistic" in words):
        d={}
        for i in data.columns:
          if(data[i].dtype=='object'):
            d[i]=data[i].unique()
        print(d)
        for i in d.keys():
          if(len(d[i])==2):
            data[i]=data[i].apply(lambda x: 0 if(x==d[i][0]) else 1)
          else:
            x=pd.get_dummies(data[i],dtype=int,prefix=i)
            data=pd.concat([data,x],axis=1)
            data.drop(i,axis=1,inplace=True)
            x=0
        m=agent.run(input=f"answer in 1 word. Which among the 'Normalization/Standardization' will be best suited for {data}",handle_parsing_errors=True)
        print(m)
        x=m.split()
        col=[]
        if('Normalization' in x):
          from sklearn.preprocessing import MinMaxScaler
          sc=MinMaxScaler()
          for i in data.columns:
            if(data[i].dtype=='object'):
              pass
            else:
              col.append(i)
          data[col]=sc.fit_transform(data[col])
        else:
          from sklearn.preprocessing import StandardScaler
          sc=StandardScaler()
          for i in data.columns:
            if(data[i].dtype=='object'):
              pass
            else:
              col.append(i)
          data[col]=sc.fit_transform(data[col])
        from sklearn.feature_selection import RFE
        from sklearn.model_selection import train_test_split
        from sklearn.linear_model import LogisticRegression
        from sklearn.metrics import confusion_matrix,accuracy_score,classification_report
        x_train,x_test,y_train,y_test=train_test_split(data,target,test_size=0.2,random_state=42)
        accuracy_list={}
        x=int(input(f'''Enter the choice for classification model\n1. Logistic Regression\n2. Naive Bayes \n3.K-Neighbour Model\n4.Decision Tree\n5. No choice'''))
        if(x==1):
          lr=LogisticRegression()
          lr.fit(x_train,y_train)
          y_pred=lr.predict(x_test)
          print(confusion_matrix(y_test,y_pred))
          accuracy_list['Logistic_Regression']=accuracy_score(y_test,y_pred)
          print(accuracy_score(y_test,y_pred))
          print(classification_report(y_test,y_pred))
        if(x==2):
          from sklearn.naive_bayes import GaussianNB
          nb=GaussianNB()
          nb.fit(x_train,y_train)
          y_pred=nb.predict(x_test)
          print(confusion_matrix(y_test,y_pred))
          accuracy_list['Naive_Bayes']=accuracy_score(y_test,y_pred)
          print(accuracy_score(y_test,y_pred))
          print(classification_report(y_test,y_pred))
        if(x==3):
          from sklearn.neighbors import KNeighborsClassifier
          knn=KNeighborsClassifier()
          knn.fit(x_train,y_train)
          y_pred=knn.predict(x_test)
          print(confusion_matrix(y_test,y_pred))
          accuracy_list['KNN']=accuracy_score(y_test,y_pred)
          print(accuracy_score(y_test,y_pred))
          print(classification_report(y_test,y_pred))
        if(x==4):
          from sklearn.tree import DecisionTreeClassifier
          dt=DecisionTreeClassifier()
          dt.fit(x_train,y_train)
          y_pred=dt.predict(x_test)
          print(confusion_matrix(y_test,y_pred))
          accuracy_list['Decision_tree']=accuracy_score(y_test,y_pred)
          print(accuracy_score(y_test,y_pred))
          print(classification_report(y_test,y_pred))
        if(x==5):
          lr=LogisticRegression()
          lr.fit(x_train,y_train)
          y_pred=lr.predict(x_test)
          accuracy_list['Logistic_Regression']=accuracy_score(y_test,y_pred)
          from sklearn.naive_bayes import GaussianNB
          nb=GaussianNB()
          nb.fit(x_train,y_train)
          y_pred=nb.predict(x_test)
          accuracy_list['Naive_Bayes']=accuracy_score(y_test,y_pred)
          from sklearn.neighbors import KNeighborsClassifier
          knn=KNeighborsClassifier()
          knn.fit(x_train,y_train)
          y_pred=knn.predict(x_test)
          accuracy_list['KNN']=accuracy_score(y_test,y_pred)
          from sklearn.tree import DecisionTreeClassifier
          dt=DecisionTreeClassifier()
          dt.fit(x_train,y_train)
          y_pred=dt.predict(x_test)
          accuracy_list['Decision_tree']=accuracy_score(y_test,y_pred)
          sorted_dict = dict(sorted(accuracy_list.items(), key=lambda item: item[1]))
          print("Sorted Dictionary:", sorted_dict)
          i = list(sorted_dict.keys())[0]
          if(i=='Logistic_Regression'):
              lr=LogisticRegression()
              lr.fit(x_train,y_train)
              y_pred=lr.predict(x_test)
              print(confusion_matrix(y_test,y_pred))
              print(accuracy_score(y_test,y_pred))
              print(classification_report(y_test,y_pred))
          if(i=='Naive_Bayes'):
              from sklearn.naive_bayes import GaussianNB
              nb=GaussianNB()
              nb.fit(x_train,y_train)
              y_pred=nb.predict(x_test)
              print(confusion_matrix(y_test,y_pred))
              print(accuracy_score(y_test,y_pred))
              print(classification_report(y_test,y_pred))
          if(i=='KNN'):
              from sklearn.neighbors import KNeighborsClassifier
              knn=KNeighborsClassifier()
              knn.fit(x_train,y_train)
              y_pred=knn.predict(x_test)
              print(confusion_matrix(y_test,y_pred))
              print(accuracy_score(y_test,y_pred))
              print(classification_report(y_test,y_pred))
          if(i=='Decision_tree'):
              from sklearn.tree import DecisionTreeClassifier
              dt=DecisionTreeClassifier()
              dt.fit(x_train,y_train)
              y_pred=dt.predict(x_test)
              print(confusion_matrix(y_test,y_pred))
              print(accuracy_score(y_test,y_pred))
              print(classification_report(y_test,y_pred))






    analyse(data)


execute_model()

# Create nbformat notebook
nb = nbformat.v4.new_notebook()

# Append executed lines to notebook cells
executed_lines = tracer.results()

# **Change:** Access the executed lines using the `files` attribute
# and iterating through the lines of each file.
executed_lines_list = []
for filename in executed_lines.files:
    for lineno in executed_lines.files[filename]:
        executed_lines_list.append(str(lineno))

nb.cells.append(nbformat.v4.new_code_cell('\n'.join(executed_lines_list)))

# Save as .ipynb file
with open("executed_lines.ipynb", "w") as f:
    nbformat.write(nb, f)

<ipython-input-3-4ac89d7481a7>:37: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  model = ChatOpenAI(
<ipython-input-3-4ac89d7481a7>:52: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. See LangGraph documentation for more details: https://langchain-ai.github.io/langgraph/. Refer here for its pre-built ReAct agent: https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/
  agent = initialize_agent(
<ipython-input-



> Entering new AgentExecutor chain...
 To answer this question, I need to gather information about the dataset using the column names, the first 5 rows, mode values, and null value counts. I will use the ChatGPT tool to get this information. Then, I will suggest the best method to handle the null values based on the dataset's characteristics.

Action: ChatGPT
Action Input: Column names, first 5 rows, and dataset information request for: age, gender, fever, cough, city, has_covid


<ipython-input-3-4ac89d7481a7>:45: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = model.predict(query)



Observation:  Here is an example of how the data you requested might look:

| age | gender | fever | cough | city     | has\_covid |
|-----|--------|-------|-------|----------|------------|
| 35  | Male   | No    | Yes   | New York | No         |
| 27  | Female | Yes   | Yes   | Chicago  | Yes        |
| 42  | Male   | No    | No    | Los Angeles  | No         |
| 54  | Female | Yes   | No    | Miami    | Yes        |
| 48  | Male   | No    | Yes   | New York | No         |

Dataset information:

* This dataset contains information about 1000 individuals, including their age, gender, whether they have a fever and/or cough, the city they are from, and whether they have been diagnosed with COVID-19.
* The age column contains integer values representing the age of each individual.
* The gender column contains either "Male" or "Female" indicating the gender of each individual.
* The fever and cough columns contain either "Yes" or "No" indicating whether the individual has a fever or cough

KeyboardInterrupt: Interrupted by user

In [4]:
import trace
import nbformat

# Initialize a tracer
tracer = trace.Trace(count=False, trace=True)

# Function to log executed lines
def trace_function(func):
    def wrapper(*args, **kwargs):
        tracer.runfunc(func, *args, **kwargs)
        return tracer.results()
    return wrapper

# Code Execution
@trace_function
def execute_model():
    # Your code goes here
    import re
    import os
    import string
    import numpy as np
    import pandas as pd
    from langchain.agents import initialize_agent, Tool
    from langchain.agents import AgentType

    data = pd.read_csv('/content/covid_toy.csv')

    # Define the provided code as is
    def analyse(data):

      os.environ['TOGETHER_API_KEY'] = "666e8dc46b45912071558831227d603654035a8f70c3e9f0a41f6f0f9661ad98"
      serpapi='db7f6d98016782c4c1d09f3b05d83bc82b64ec5f64a9473c573b111c723a3f64'
      model = ChatOpenAI(
          base_url="https://api.together.xyz/v1",
          api_key=os.environ["TOGETHER_API_KEY"],
          model="mistralai/Mixtral-8x7B-Instruct-v0.1",
      )

      def chatgpt_tool(query: str) -> str:
          """Use ChatGPT to answer general knowledge questions."""
          response = model.predict(query)
          return response
      # Import Tool from langchain.agents
      tools = [
          Tool(name="ChatGPT", func=chatgpt_tool, description="Answers general questions using ChatGPT.")
      ]

      agent = initialize_agent(
          tools,
          model,
          agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
          verbose=True,
      )
      prompt=f"Describe the dataset {data} using its column names, the first 5 rows, mode values, and null value counts. Suggest the best method to handle the null values."
      out=agent.run(input=prompt,handle_parsing_errors=True)
      print(out)
      value=[]
      num=[i for i in range(1,11)]
      floats=[float(i) for i in num]
      symbols =[',','/',';',':','%' ,'$','&','<','>','*','^','(',')','#','@','!','-1','?']
      alphabet=list(string.ascii_lowercase)
      for i in data.columns:
        if isinstance(data[i].dtype , object):
                  x=data[i].mode()[0]
                  value.append(x)
                  index=len(data)//5
                  for m in range (1,6):
                      if index * m < len(data):
                        x=data.iloc[index*m][i]
                      else:
                        x=data.iloc[len(data)-1][i]

                      value.append(x)
                  l=[]
                  count=0
                  for m in value:
                    x=list(str(m))
                    for j in x:
                      if (j not in alphabet):
                          l.append(j)
                    if(len(x)==len(l)):
                      count+=1
                      l=[]
                  if(count==6):
                    for m in value:
                      x=list(str(m))
                      for j in x:
                            if(j in symbols):
                                data[i]=data[i].apply(lambda x: x.replace(j,''))
                    data[i]=data[i].astype(float)
                    print(data[i].dtype)

      for i in symbols:
          data.replace({i: np.nan}, inplace=True)
      col=list(data.columns[data.isnull().any()])
      for i in col:
          print(data[i].dtype)
          if any(letter in str(val) for val in data[i] for letter in alphabet):
              x=int(input(f'''For column {i} Enter the choice for null value handling\n1. Deletion\n2. Fill with Mode value\n3. Fill with Median\n4.Fill with Mean\n5. Fill with the mean of last 10 non-null values\n6.Fill with the mean of last 20 non-null values'''))
              if(x==1):
                      data[i].dropna(inplace=True)
              if(x==2):
                      data[i].fillna(data[i].mode()[0],inplace=True)
              if(x==3):
                      data[i].fillna(data[i].median(),inplace=True)
              if(x==4):
                      data[i].fillna(data[i].mean(),inplace=True)
              if(x==5):
                      data[i].fillna(data[i].rolling(10,min_periods=1).mean(),inplace=True)
              if(x==6):
                      data[i].fillna(data[i].rolling(20,min_periods=1).mean(),inplace=True)


          else:
              x=int(input(f'''For column {i} Enter the choice for null value handling\n1. Deletion\n2. Fill with Mode value\n3. Fill with Median\n4.Fill with Mean\n5. Fill with the mean of last 10 non-null values\n6. Fill with the mean of last 20 non-null values'''))
              if(x==1):
                      data[i].dropna(inplace=True)
              if(x==2):
                      data[i].fillna(data[i].mode()[0],inplace=True)
              if(x==3):
                      data[i].fillna(data[i].median(),inplace=True)
      # data modeling
      prompt=f'From the dataset {data}, identify the target column and specify whether to use linear or logistic regression. Only provide the column name and regression type.'
      output=agent.run(input=prompt,handle_parsing_errors=True)
      print(output)
      words= re.split(r'[,\s\'\"]+', output)
      print(words)
      for i in data.columns:
        if(i in words):
          print(i)
          target=data.pop(i)
        else:
          pass
      if('linear' in words) or ("Linear" in words):
        d={}
        for i in data.columns:
          if(data[i].dtype=='object'):
            d[i]=data[i].unique()
        print(d)
        for i in d.keys():
          if(len(d[i])==2):
            data[i]=data[i].apply(lambda x: 0 if(x==d[i][0]) else 1)
          else:
            x=pd.get_dummies(data[i],dtype=int,prefix=i)
            data=pd.concat([data,x],axis=1)
            data.drop(i,axis=1,inplace=True)
            x=0
        m=agent.run(input=f"Which among the 'Normalization/Standardization' will be best suited for {data}, use only 20 characters for answer",handle_parsing_errors=True)
        print(m)
        x=m.split()
        col=[]
        if('Normalization' in x):
          from sklearn.preprocessing import MinMaxScaler
          sc=MinMaxScaler()
          for i in data.columns:
            if(data[i].dtype=='object'):
              pass
            else:
              col.append(i)
          data[col]=sc.fit_transform(data[col])
        else:
          from sklearn.preprocessing import StandardScaler
          sc=StandardScaler()
          for i in data.columns:
            if(data[i].dtype=='object'):
              pass
            else:
              col.append(i)
          data[col]=sc.fit_transform(data[col])
        from sklearn.feature_selection import RFE
        from sklearn.model_selection import train_test_split
        from sklearn.linear_model import LinearRegression
        from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error,root_mean_squared_error
        x_train,x_test,y_train,y_test=train_test_split(data,target,test_size=0.2,random_state=42)
        lr=LinearRegression()
        lr.fit(x_train,y_train)
        y_pred=lr.predict(x_test)
        print('r2 : ' ,r2_score(y_test,y_pred))
        print('MSE : ' ,mean_squared_error(y_test,y_pred))
        print('RMSE : ' ,root_mean_squared_error(y_test,y_pred))
        print('MAE : ' ,mean_absolute_error(y_test,y_pred))
        print(data)

      elif('logistic' in words) or ("Logistic" in words):
        d={}
        for i in data.columns:
          if(data[i].dtype=='object'):
            d[i]=data[i].unique()
        print(d)
        for i in d.keys():
          if(len(d[i])==2):
            data[i]=data[i].apply(lambda x: 0 if(x==d[i][0]) else 1)
          else:
            x=pd.get_dummies(data[i],dtype=int,prefix=i)
            data=pd.concat([data,x],axis=1)
            data.drop(i,axis=1,inplace=True)
            x=0
        m=agent.run(input=f"answer in 1 word. Which among the 'Normalization/Standardization' will be best suited for {data}",handle_parsing_errors=True)
        print(m)
        x=m.split()
        col=[]
        if('Normalization' in x):
          from sklearn.preprocessing import MinMaxScaler
          sc=MinMaxScaler()
          for i in data.columns:
            if(data[i].dtype=='object'):
              pass
            else:
              col.append(i)
          data[col]=sc.fit_transform(data[col])
        else:
          from sklearn.preprocessing import StandardScaler
          sc=StandardScaler()
          for i in data.columns:
            if(data[i].dtype=='object'):
              pass
            else:
              col.append(i)
          data[col]=sc.fit_transform(data[col])
        from sklearn.feature_selection import RFE
        from sklearn.model_selection import train_test_split
        from sklearn.linear_model import LogisticRegression
        from sklearn.metrics import confusion_matrix,accuracy_score,classification_report
        x_train,x_test,y_train,y_test=train_test_split(data,target,test_size=0.2,random_state=42)
        accuracy_list={}
        x=int(input(f'''Enter the choice for classification model\n1. Logistic Regression\n2. Naive Bayes \n3.K-Neighbour Model\n4.Decision Tree\n5. No choice'''))
        if(x==1):
          lr=LogisticRegression()
          lr.fit(x_train,y_train)
          y_pred=lr.predict(x_test)
          print(confusion_matrix(y_test,y_pred))
          accuracy_list['Logistic_Regression']=accuracy_score(y_test,y_pred)
          print(accuracy_score(y_test,y_pred))
          print(classification_report(y_test,y_pred))
        if(x==2):
          from sklearn.naive_bayes import GaussianNB
          nb=GaussianNB()
          nb.fit(x_train,y_train)
          y_pred=nb.predict(x_test)
          print(confusion_matrix(y_test,y_pred))
          accuracy_list['Naive_Bayes']=accuracy_score(y_test,y_pred)
          print(accuracy_score(y_test,y_pred))
          print(classification_report(y_test,y_pred))
        if(x==3):
          from sklearn.neighbors import KNeighborsClassifier
          knn=KNeighborsClassifier()
          knn.fit(x_train,y_train)
          y_pred=knn.predict(x_test)
          print(confusion_matrix(y_test,y_pred))
          accuracy_list['KNN']=accuracy_score(y_test,y_pred)
          print(accuracy_score(y_test,y_pred))
          print(classification_report(y_test,y_pred))
        if(x==4):
          from sklearn.tree import DecisionTreeClassifier
          dt=DecisionTreeClassifier()
          dt.fit(x_train,y_train)
          y_pred=dt.predict(x_test)
          print(confusion_matrix(y_test,y_pred))
          accuracy_list['Decision_tree']=accuracy_score(y_test,y_pred)
          print(accuracy_score(y_test,y_pred))
          print(classification_report(y_test,y_pred))
        if(x==5):
          lr=LogisticRegression()
          lr.fit(x_train,y_train)
          y_pred=lr.predict(x_test)
          accuracy_list['Logistic_Regression']=accuracy_score(y_test,y_pred)
          from sklearn.naive_bayes import GaussianNB
          nb=GaussianNB()
          nb.fit(x_train,y_train)
          y_pred=nb.predict(x_test)
          accuracy_list['Naive_Bayes']=accuracy_score(y_test,y_pred)
          from sklearn.neighbors import KNeighborsClassifier
          knn=KNeighborsClassifier()
          knn.fit(x_train,y_train)
          y_pred=knn.predict(x_test)
          accuracy_list['KNN']=accuracy_score(y_test,y_pred)
          from sklearn.tree import DecisionTreeClassifier
          dt=DecisionTreeClassifier()
          dt.fit(x_train,y_train)
          y_pred=dt.predict(x_test)
          accuracy_list['Decision_tree']=accuracy_score(y_test,y_pred)
          sorted_dict = dict(sorted(accuracy_list.items(), key=lambda item: item[1]))
          print("Sorted Dictionary:", sorted_dict)
          i = list(sorted_dict.keys())[0]
          if(i=='Logistic_Regression'):
              lr=LogisticRegression()
              lr.fit(x_train,y_train)
              y_pred=lr.predict(x_test)
              print(confusion_matrix(y_test,y_pred))
              print(accuracy_score(y_test,y_pred))
              print(classification_report(y_test,y_pred))
          if(i=='Naive_Bayes'):
              from sklearn.naive_bayes import GaussianNB
              nb=GaussianNB()
              nb.fit(x_train,y_train)
              y_pred=nb.predict(x_test)
              print(confusion_matrix(y_test,y_pred))
              print(accuracy_score(y_test,y_pred))
              print(classification_report(y_test,y_pred))
          if(i=='KNN'):
              from sklearn.neighbors import KNeighborsClassifier
              knn=KNeighborsClassifier()
              knn.fit(x_train,y_train)
              y_pred=knn.predict(x_test)
              print(confusion_matrix(y_test,y_pred))
              print(accuracy_score(y_test,y_pred))
              print(classification_report(y_test,y_pred))
          if(i=='Decision_tree'):
              from sklearn.tree import DecisionTreeClassifier
              dt=DecisionTreeClassifier()
              dt.fit(x_train,y_train)
              y_pred=dt.predict(x_test)
              print(confusion_matrix(y_test,y_pred))
              print(accuracy_score(y_test,y_pred))
              print(classification_report(y_test,y_pred))
        print(data)





    analyse(data)
# Replace with the actual model execution code

# Run the model and get the results
executed_lines = execute_model()

# Process executed lines
executed_lines_list = []
for filename, lines in executed_lines.counts.items():
    for lineno in lines.keys():
        executed_lines_list.append(f"{filename}: Line {lineno}")

# Create nbformat notebook
nb = nbformat.v4.new_notebook()

# Append executed lines to notebook cells
nb.cells.append(nbformat.v4.new_code_cell('\n'.join(executed_lines_list)))

# Save as .ipynb file
with open("executed_lines.ipynb", "w") as f:
    nbformat.write(nb, f)

print("Executed lines saved to 'executed_lines.ipynb'")


Streaming output truncated to the last 5000 lines.
format.py(1827):         numbers = [x for x in values if is_number_with_decimal(x)]
 --- modulename: format, funcname: is_number_with_decimal
format.py(1817):         return re.match(number_regex, x) is not None
 --- modulename: re, funcname: match
re.py(190):     return _compile(pattern, flags).match(string)
 --- modulename: re, funcname: _compile
re.py(290):     if isinstance(flags, RegexFlag):
re.py(292):     try:
re.py(293):         return _cache[type(pattern), pattern, flags]
re.py(294):     except KeyError:
re.py(295):         pass
re.py(296):     if isinstance(pattern, Pattern):
re.py(297):         if flags:
re.py(300):         return pattern
format.py(1827):         numbers = [x for x in values if is_number_with_decimal(x)]
 --- modulename: format, funcname: is_number_with_decimal
format.py(1817):         return re.match(number_regex, x) is not None
 --- modulename: re, funcname: match
re.py(190):     return _compile(pattern, f